# DS7331 Project 3
#### Group 2: Hollie Gardner, Cleveland Johnson, Shelby Provost
[Dataset Source](https://archive-beta.ics.uci.edu/ml/datasets/census+income)<br/>
[Github Repo](https://github.com/ShelbyP27/DS7331-Project)

## Import Libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import sklearn as sk

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# data preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

#prediction models
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.neighbors import KNeighborsClassifier

# cluster analysis

# arm
# https://mhahsler.github.io/arules/docs/python/arules_python.html
from rpy2.robjects import pandas2ri
pandas2ri.activate()

import rpy2.robjects as ro
from rpy2.robjects.packages import importr

arules = importr("arules")
arulesViz = importr("arulesViz")

import time
%matplotlib inline

%load_ext rmagic
%load_ext rpy2.ipython 



/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/rmagic.py:11: UserWarning: The rmagic extension in IPython has moved to `rpy2.ipython`, please see `rpy2` documentation.
  warnings.warn("The rmagic extension in IPython has moved to "


## Data Preparation

### Loading and Prepping Data 


In [2]:
# Importing the census dataset using pandas
# Reading the CSV file after converting file to csv and removing superfluous spaces via Excel.
df = pd.read_csv('https://raw.githubusercontent.com/ShelbyP27/DS7331-Project/main/adult-data.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
#Cleaning up dataset by removing na values & creating one-hot response
df = df.replace(to_replace='?',value=np.nan) # replace '?' with NaN (not a number)
df.dropna(inplace=True) # Removing na values
df.duplicated(subset=None, keep='first') #Remove duplicates

#change weird value for income
df['income'] = df['income'].replace(['<=50K'],'under50K')
df['income'] = df['income'].replace(['>50K'],'over50K')



In [4]:
#remove attributes
del df['education-num']
del df['fnlwgt']
del df['capital-gain']
del df['capital-loss']

#create categorical groups out of continuous
#change age to age_group
age_group = pd.cut(df.age,bins=[12,19,34,65,120], labels=['teenager','early adult','adult','older adult'])
df.insert(2,'agegroup',age_group)
del df['age']

#change hours-per-week to 
hours_group = pd.cut(df['hours-per-week'], bins=[0,30,40,168], labels=['part-time','full-time','over-time'])
df.insert(9,'hoursgroup',hours_group)
del df['hours-per-week']



## Business Understanding
*Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

The purpose of the dataset selected is to identify what characteristics an individual needs to possess in order to make an income exceeding $50,000. Since we've chosen to analyze using association rule mining, then measurements of effectiveness will be lift and [insert here] when we filter to our question of interest of whether income is greater or less than $50k is on the right hand side. The algorithm must also not be a burden computational, therefore we will adjust our confidence and support requirements. The length of itemlists on the left-hand-side will ideally be greater than 1. 

We have chosen to use the association rule mining by treating each individual and their characteristics as a transaction. The "item lists", or characteristics of the individuals within this dataset, will hopefully accomplish the task of understanding what are the most important characteristics that determine whether someone in the 1990s United States would have had an income that exceed $50k. 

Knowing this information could assist in targeted support for families that had less than $50k and figuring out what interventions could be used to bring families out of poverty levels and into liveable income. 



## Data Understanding
### Part One
*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?*

The final dataset that will be used for the association rule minging of individuals with an income greater than 50k includes the following 10 attributes: 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   workclass       30162 non-null  object  
 1   agegroup        30162 non-null  category
 2   education       30162 non-null  object  
 3   marital-status  30162 non-null  object  
 4   occupation      30162 non-null  object  
 5   relationship    30162 non-null  object  
 6   race            30162 non-null  object  
 7   sex             30162 non-null  object  
 8   hoursgroup      30162 non-null  category
 9   native-country  30162 non-null  object  
 10  income          30162 non-null  object  
dtypes: category(2), object(9)
memory usage: 2.4+ MB


All of the continuous variables have either been removed or converted into categorical groups for easier identification in the transaction lists. 

According to the Kohavi and Becker, the following changes have already been made to the raw data:
* Discretized agrossincome into two ranges with threshold 50,000.
* Convert U.S. to US to avoid periods.
* Convert Unknown to "?"
* Run MLC++ GenCVFiles to generate data,test.
<br>

Therefore, we began by looking for the '?' values in the dataset by counting the number in each column. Then, we replaced as null using numpy. We decided to handle missing values in the following three variables which possessed missing values as below: 

* Workclass: 1836 rows
* Occupation: 1843 rows
* Native-Country: 583 rows

There was not a way to impute values for these categorical variables and leaving as unknown variables does not add value to our analysis. Therefore, these rows were removed leaving us with 30,162 complete rows in this dataframe. As for duplicates, Kohavi and Becker described 6 duplicate rows. After reviewing our current dataframe, there do not appear to be any instances of duplicate data.


### Part Two
*Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.*

----INSERT REVIEW OF ATTRIBUTE SELECTION PROCESS. MAY NEED TO REVISIT #1 & #2----

## Modeling and Evaluation
*Different tasks will require different evaluation methods. Be as thorough as possible when analyzing the data you have chosen and use visualizations of the results to explain the performance and expected outcomes whenever possible. Guide the reader through your analysis with plenty of discussion of the results.*
 

### Option A: Cluster Analysis

 - Perform cluster analysis using several clustering methods
 - How did you determine a suitable number of clusters for each method?
 - Use internal and/or external validation measures to describe and compare the clusterings and the clusters (some visual methods would be good).
 - Describe your results. What findings are the most interesting and why? 


#### Data prep for cluster analysis

In [ ]:
# Separating the features from the response
if 'income' in df:
    y = df['income'].values
    del df['income']
    X = df.values

# Train / Test split with scaled_X
scaled_X = StandardScaler().fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(scaled_X, y, test_size = .2, random_state=1)

In [ ]:
# creating a reduced version of dataframe with fewer columns
df_reduced = df[['age','hours-per-week', 'capital-loss','education-num','Marital_Never-married','Marital_Married-civ-spouse','Work_Private', 'Rel_Husband','capital-gain','Rel_Not-in-family']].values

df_red_scaled = StandardScaler().fit_transform(df_reduced)
x_train, x_test, y_train, y_test = train_test_split(df_red_scaled, y, test_size = .2, random_state=1)


#### Kmeans Cluster Analysis

In [ ]:
#kmeans
from sklearn.cluster import KMeans

X1 = df_red_scaled

# run kmeans algorithm (this is the most traditional use of k-means)
kmeans = KMeans(init='random', # initialization
        n_clusters=100,  # number of clusters
        n_init=5)       # number of different times to run k-means
       
kmeans.fit(X1)

# visualize the data
centroids = kmeans.cluster_centers_
plt.plot(X1[:, 0], X1[:, 1], 'r.', markersize=2) #plot the data
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='+', s=200, linewidths=3, color='k')  # plot the centroids
plt.title('K-means clustering for X1')
plt.xlabel('X1, Feature 1')
plt.ylabel('X1, Feature 2')
plt.grid()
plt.show()

In [ ]:
from sklearn.cluster import MiniBatchKMeans

kmeans_mini = MiniBatchKMeans(n_clusters=2, batch_size=1000)
kmeans = KMeans(n_clusters=2)

print('Time for BatchKMeans:')
%time kmeans.fit(X1)
print('Time for MiniBatchKMeans:')
%time kmeans_mini.fit(X1)


# visualize the data
centroids = kmeans.cluster_centers_
plt.plot(X1[:, 0], X1[:, 1], 'r.', markersize=2) #plot the data
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='+', s=200, linewidths=3, label='Batch')  # plot the centroids

centroids = kmeans_mini.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='+', s=200, linewidths=3, color='k',label='Mini-Batch')  # plot the centroids
plt.legend()
plt.title('K-means clustering for X1')
plt.xlabel('X1, Feature 1')
plt.ylabel('X1, Feature 2')
plt.grid()
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN

# now plot each dataset
plt.figure(figsize=(15,5))
for i,X in enumerate([X1]):
    plt.subplot(1,2,i+1)
    plt.plot(X[:, 0], X[:, 1], 'r.', markersize=2) #plot the data
    plt.title('Variable name: X{0}'.format(i+2))
    plt.xlabel('X{0}, Feature 1'.format(i+2))
    plt.ylabel('X{0}, Feature 2'.format(i+2))
    plt.grid()

plt.show()

In [ ]:
# lets first look at the connectivity of the graphs and distance to the nearest neighbors
from sklearn.neighbors import kneighbors_graph
X2 = X1

#=======================================================
# CHANGE THESE VALUES TO ADJUST MINPTS FOR EACH DATASET
X2_N = 5000

#=======================================================

# create connectivity graphs before calcualting the hierarchy
X2_knn_graph = kneighbors_graph(X2, X2_N, mode='distance') # calculate distance to four nearest neighbors 

N2 = X2_knn_graph.shape[0]
X2_4nn_distances = np.zeros((N2,1))
for i in range(N2):
    X2_4nn_distances[i] = X2_knn_graph[i,:].max()

X2_4nn_distances = np.sort(X2_4nn_distances, axis=0)


plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(range(N2), X2_4nn_distances, 'r.', markersize=2) #plot the data
plt.title('Dataset name: X2, sorted by neighbor distance')
plt.xlabel('X2, Instance Number')
plt.ylabel('X2, Distance to {0}th nearest neighbor'.format(X2_N))
plt.grid()

plt.show()

### Option B: Association Rule Mining
 - Create frequent itemsets and association rules.
 - Use tables/visualization to discuss the found results.
 - Use several measure for evaluating how interesting different rules are.
 - Describe your results. What findings are the most compelling and why? 

### Step One
We begin with a generic support set at .5 and confidence at .8. The maxlen is set at 5 to reduce computational load.

In [6]:
# run apriori on dataframe
%R -i df rules <- apriori(df,parameter = list(minlen=2, maxlen=5, supp=0.5, conf=0.8))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.8    0.1    1 none FALSE            TRUE       5     0.5      2
 maxlen target  ext
      5  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 15081 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[107 item(s), 30162 transaction(s)] done [0.05s].
sorting and recoding items ... [7 item(s)] done [0.00s].
creating transaction tree ... done [0.02s].
checking subsets of size 1 2 3 done [0.00s].
writing ... [15 rule(s)] done [0.00s].
creating S4 object  ... done [0.01s].


<rpy2.robjects.methods.RS4 object at 0x12aa23640> [RTYPES.S4SXP]
R classes: ('rules',)

In [7]:
#looking at the top rules
%R inspect(head(rules, n = 10, by = "lift"))

     lhs                                rhs                              support confidence  coverage      lift count
[1]  {sex=Male,                                                                                                      
      native-country=United-States}  => {race=White}                   0.5585836  0.9071721 0.6157417 1.0551083 16848
[2]  {sex=Male}                      => {race=White}                   0.5980373  0.8850834 0.6756846 1.0294176 18038
[3]  {race=White,                                                                                                    
      sex=Male}                      => {native-country=United-States} 0.5585836  0.9340282 0.5980373 1.0242931 16848
[4]  {race=White}                    => {native-country=United-States} 0.8029308  0.9338680 0.8597905 1.0241175 24218
[5]  {native-country=United-States}  => {race=White}                   0.8029308  0.8805265 0.9118759 1.0241175 24218
[6]  {workclass=Private,                                

We see the top 10 rules ordered by the highest lift. These rules seem to highlight the large number of Males from the United States who are represented in this dataset. 

Next, we will limit to rules that have income attribute representing all levels within the right-hand-set (rhs)

In [ ]:
# Keeps getting a recursion error
#%R rules.sorted <- sort(rules, by='lift')
#%R plot(rules.sorted, method='grouped')

In [21]:
## find only itemsets with income attribute in the right-hand-side.
# from https://cran.r-project.org/web/packages/arules/arules.pdf 
%R rules <- apriori(df, parameter = list(minlen=1, maxlen=10, supp=0.05, conf=0.5), appearance = list(items = c("income=over50K", "income=under50K")))
%R inspect(head(rules))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.5    0.1    1 none FALSE            TRUE       5    0.05      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 1508 

set item appearances ...[2 item(s)] done [0.01s].
set transactions ...[2 item(s), 30162 transaction(s)] done [0.11s].
sorting and recoding items ... [2 item(s)] done [0.00s].
creating transaction tree ... done [0.01s].
checking subsets of size 1 done [0.00s].
writing ... [0 rule(s)] done [0.01s].
creating S4 object  ... done [0.02s].


In [17]:
#looking at the top rules
%R inspect(head(rules, n = 10, by = "lift"))

## Deployment
 - Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling?
 - How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)?
 - How would your deploy your model for interested parties?
 - What other data should be collected?
 - How often would the model need to be updated, etc.? 

## Exceptional Work
 - You have free reign to provide additional analyses or combine analyses 